In [1]:
import pandas as pd
import sqlite3

#incarcam datele
results = pd.read_csv("results.csv")
shootouts = pd.read_csv("shootouts.csv")

In [33]:
def connect_to_database(database_name):
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()
    return conn, cursor

# Conectare la baza de date
conn, cursor = connect_to_database('football.db')

results.to_sql('results',conn, if_exists='replace', index=False)
shootouts.to_sql('shootouts', conn, if_exists='replace', index=False)

# Adăugare coloane 'day', 'month' și 'year' în tabela 'results', dacă nu există deja
cursor.execute("""
    PRAGMA table_info(results);
""")
columns = cursor.fetchall()
column_names = [column[1] for column in columns]
if 'day' not in column_names:
    cursor.execute("""
        ALTER TABLE results
        ADD COLUMN day INTEGER;
    """)
if 'month' not in column_names:
    cursor.execute("""
        ALTER TABLE results
        ADD COLUMN month INTEGER;
    """)
if 'year' not in column_names:
    cursor.execute("""
        ALTER TABLE results
        ADD COLUMN year INTEGER;
    """)

# Adăugare coloane 'day', 'month' și 'year' în tabela 'shootouts', dacă nu există deja
cursor.execute("""
    PRAGMA table_info(shootouts);
""")
columns = cursor.fetchall()
column_names = [column[1] for column in columns]
if 'day' not in column_names:
    cursor.execute("""
        ALTER TABLE shootouts
        ADD COLUMN day INTEGER;
    """)
if 'month' not in column_names:
    cursor.execute("""
        ALTER TABLE shootouts
        ADD COLUMN month INTEGER;
    """)
if 'year' not in column_names:
    cursor.execute("""
        ALTER TABLE shootouts
        ADD COLUMN year INTEGER;
    """)


# Salvează modificările în baza de date
conn.commit()

#inchidem conexiunea la baza de date
conn.close()

In [62]:
import re
import sqlite3

def extract_date_info(date_str):
    try:
        month, day, year = map(int, date_str.split('/'))
        if year >= 1900:
            return day, month, year
    except ValueError:
        pass

    match = re.match(r'(\d{1,2})-(\d{1,2})-(\d{4})', date_str)
    if match:
        groups = match.groups()
        month, day, year = map(int, (groups[0], groups[1], groups[2]))
        return day, month, year

    match = re.match(r'(\d{4})-(\d{2})-(\d{2})', date_str)
    if match:
        groups = match.groups()
        year, month, day = map(int, (groups[0], groups[1], groups[2]))
        return day, month, year

    return None, None, None

# Conectare la baza de date
conn = sqlite3.connect('football.db')
cursor = conn.cursor()

# Actualizează valorile pentru coloanele 'day', 'month', 'year' folosind datele extrase
cursor.execute("SELECT date FROM results;")
dates = cursor.fetchall()

for date_row in dates:
    date_str = date_row[0]
    day, month, year = extract_date_info(date_str)
    cursor.execute("UPDATE results SET day=?, month=?, year=? WHERE date=?", (day, month, year, date_str))

conn.commit() # Salvează modificarile în baza de date

# Închide conexiunea la baza de date
conn.close()